# Projet 6 : Améliorez le produit IA de votre start-up


<center><img alt="aliment" src="images/ia_repas.png" height=500 width=500/></center>


# 1. INTRODUCTION
## Contexte

Contexte : Avis Restau est une plateforme permettant de mettre en relation des clients et des restaurants. Elle permet aux clients de poster leurs avis, photos et des notes sur les restaurants qu’ils fréquentent.

<center><img alt="avis_resto" src="images/avis_resto.png"/></center>
    
## Objectifs 
1. Détecter les sujets d’insatisfaction présents dans les commentaires postés sur la plateforme.<br>
2. Labelliser automatiquement les photos postées sur la plateforme.

# 2. Démarche méthodologique

Dans un premier temps, je vais <a href="Renard_Anthony_2_notebook_022023.ipynb#C313">charger</a> les fichiers sources qui seront notre point d'entrée (Texte : 2 dataframes au format JSON + via API YELP; images : le package IMAGE fourni + via API YELP). Après un rassemblement/agrégation des données, je ferai un **inventaire** et un **nettoyage** des jeux de données afin de récupérer un dataset "propre" pour l'analyse des données. Finalement, je ferai la détection et l'analyse des sujets d'insatisfaction via 3 traitements des données textuelles et pour finir je ferai une labellisation automatique des photos afin d'identifier les photos relatives à la nourriture, au décor dans le restaurant ou à l’extérieur du restaurant


<center><img alt="demarche" src="images/demarche.png"/></center>

# 3. Exploration des données

## <a href="Renard_Anthony_2_notebook_022023.ipynb#C31">3.1 Chargement<a>

Chargement des librairies et des fonctions utiles. 

### Les données textuelles

Chargement des données fournies par YELP via les fichiers JSON, des informations pertinentes via l'API YELP

<center><img alt="avis_resto" src="images/chargement.png"/></center>
    


Remarque : seulement 2/5 fichier JSON ont étés utilisés : ce sont les 2 fichiers contenant les informations nécessaires pour traiter notre problématique.

Afin de récupérer les informations via l'API, j'ai créé une clé sur le site YELP afin d'avoir une autorisation d'accès à la base de données. Ensuite j'ai formulé une requête pour récupérer les noms des restaurants, les avis textuels et les notes associées. Attention : en utilisateur "lambda", je ne pouvais récupérer par requête que l'information de 50 restaurants et 3 extraits d'avis maximum par restaurant. J'ai donc fait au total 4 requêtes (via 4 fonctions) pour récupérer les avis/notes : 600 avis pour 200 restaurants.
Un fichier CSV a été crée pour stocker les informations récupérées via l'API YELP (utilisation de GRAPH GL) : ce fichier est nommé : <a href="Renard_Anthony_1_csv_022023.csv#C31">Renard_Anthony_1_csv_022023.csv<a>

### Les images

Chargement de la dataframe au format JSON et des 200098 images (qui me permet de déduire une liste)

<center><img alt="chargement" src="images/chargement2.png"/></center>

## 3.2 Présentation de la structure

###  <a href="Renard_Anthony_2_notebook_022023#C321">3.2.1 Texte : nombre de lignes et de colonnes et types<a>

Nous avons en notre possession 9 dataframes. Voici leur configuration après leur <a href="Renard_Anthony_2_notebook_022023.ipynb#C313">chargement<a> : <br> 


<table>
      <tr>
        <th><center>N°</center></th>
        <th><center>Nom</center></th>
        <th><center>Nombre de lignes</center></th>
        <th><center>Nombre de colonnes</center></th>
        <th><center>Type float64</center></th>
        <th><center>Type int64</center></th>
        <th><center>Type objet</center></th>
        <th><center>Type datetime</center></th>
      </tr>
      <tr>
        <td><center>1</center></td>
        <td><center>df_business</center></td>
        <td><center>150346</center></td>
        <td><center>14</center></td>
        <td><center>3</center></td>
        <td><center>2</center></td>
        <td><center>9</center></td>
        <td><center>0</center></td>
      </tr>
       <tr>
        <td><center>2</center></td>
        <td><center>df_review</center></td>
        <td><center>X</center></td>
        <td><center>9</center></td>
        <td><center>0</center></td>
        <td><center>4</center></td>
        <td><center>4</center></td>
        <td><center>1</center></td>
      </tr>
       <tr>
        <td><center>3</center></td>
        <td><center>df_final_API</center></td>
        <td><center>600</center></td>
        <td><center>3</center></td>
        <td><center>0</center></td>
        <td><center>0</center></td>
        <td><center>3</center></td>
        <td><center>0</center></td>
      </tr>
</table><br>


Remarque : df_review est chargée avec DASK pour palier à sa taille importante >3Go : un nombre de lignes devra être choisie pour figer la DataFrame en mémoire. Ce choix sera fait stratégiquement dans la partie "Nettoyage".

###  <a href="Renard_Anthony_2_notebook_022023.ipynb#C322">3.2.2 Images : nombre de lignes et de colonnes et types<a>

Nous avons en notre possession 200098 images ainsi qu'un jeu de données donnant des informations complémentaires sur ces photos :

<table>
      <tr>
        <th><center>N°</center></th>
        <th><center>Nom</center></th>
        <th><center>Nombre de lignes</center></th>
        <th><center>Nombre de colonnes</center></th>
        <th><center>Type objet</center></th>
      </tr>
      <tr>
        <td><center>1</center></td>
        <td><center>df_photos</center></td>
        <td><center>200100</center></td>
        <td><center>4</center></td>
        <td><center>4</center></td>
      </tr>
      <tr>
        <td><center>2</center></td>
        <td><center>list images</center></td>
        <td><center>200098 noms d'images</center></td>
        <td><center>N/A</center></td>
        <td><center>N/A</center></td>
      </tr>
</table><br>

Ce jeu de données permettra de préciser les informations des 200098 photos.

## 3.3 Nettoyage

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C331"> 3.3.1 Texte<a>

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C3311"> 3.3.1.1 Sélection des lignes et des colonnes d'un point de vue "métier"<a>

La problématique étant la suivante : "détecter les sujets d’insatisfactions de restaurants", on va sélectionner les colonnes pertinentes répondant à notre problématique. On va donc ne garder que les colonnes : "catégorie", "notation" et "avis". Concernant les notations, nous allons ne garder que les notes inférieurs ou égales à 2/5 afin de ne récupérer que les retours des clients insatisfaits. Voici le statut des dataframes après ce filtrage :

<table>
      <tr>
        <th><center>N°</center></th>
        <th><center>Nom</center></th>
        <th><center>Nombre de lignes</center></th>
        <th><center>Nombre de colonnes</center></th>
        <th><center>Nombre de restaurants</center></th>
      </tr>
      <tr>
        <td><center>1</center></td>
        <td><center>df_business</center></td>
        <td><center>150 346</center></td>
        <td><center>3</center></td>
        <td><center>A définir</center></td>
      </tr>
       <tr>
        <td><center>2</center></td>
        <td><center>df_review_br</center></td>
        <td><center>1 613 801</center></td>
        <td><center>3</center></td>
        <td><center>A définir</center></td>
      </tr>
       <tr>
        <td><center>3</center></td>
        <td><center>df_final_API</center></td>
        <td><center>44 (-93%)</center></td>
        <td><center>3</center></td>
        <td><center>37</center></td>
      </tr>
</table><br>

A ce stade, "df_business" et "df_review_br" contiennent tous les types de commerces. Le filtrage des catégories et donc la définition du nombre de restaurants, se fera à l'étape suivante ("Les manquants")

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C3312"> 3.3.1.2 Les manquants<a>

Seul df_business présente des valeurs manquantes : 

<table>
      <tr>
        <th><center>N°</center></th>
        <th><center>Nom dataframe</center></th>
        <th><center>Nom colonne</center></th>
        <th><center>Quantité</center></th>
        <th><center>Pourcentage</center></th>
        <th><center>Commentaire</center></th>
      </tr>
      <tr>
        <td><center>1</center></td>
        <td><center>df_business</center></td>
        <td><center>categories</center></td>
        <td><center>103</center></td>
        <td><center>0.07 %</center></td>
        <td><center>Très faible % : suppression</center></td>
      </tr>
</table><br>

Le statut de df_business est :

<table>
      <tr>
        <th><center>N°</center></th>
        <th><center>Nom</center></th>
        <th><center>Nombre de lignes</center></th>
        <th><center>Nombre de colonnes</center></th>
        <th><center>Nombre de restaurants</center></th>
      </tr>
      <tr>
        <td><center>1</center></td>
        <td><center>df_business</center></td>
        <td><center>150 243 (-0.07%)</center></td>
        <td><center>3</center></td>
        <td><center>A définir</center></td>
      </tr>
</table><br>

Maintenant qu'il n'y a plus de manquant dans la colonne catégories, je peux garder seulement les lignes concernant les restaurants. Nous avons donc :

<table>
      <tr>
        <th><center>N°</center></th>
        <th><center>Nom</center></th>
        <th><center>Nombre de lignes</center></th>
        <th><center>Nombre de colonnes</center></th>
        <th><center>Nombre de restaurants</center></th>
      </tr>
      <tr>
        <td><center>1</center></td>
        <td><center>df_business</center></td>
        <td><center>52 286 (-65%)</center></td>
        <td><center>3</center></td>
        <td><center>52 286</center></td>
      </tr>
</table><br>

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C3313"> 3.3.1.3 Les types<a>

Seule la dataframe "df_final_api" est concernée :

<table>
      <tr>
        <th><center>N°</center></th>
        <th><center>Nom</center></th>
        <th><center>Type float64</center></th>
        <th><center>Type int64</center></th>
        <th><center>Type objet</center></th>
        <th><center>Type datetime</center></th>
      </tr>
      <tr>
        <td><center>1</center></td>
        <td><center>df_final_api</center></td>
        <td><center>0</center></td>
        <td><center>0 -> 1</center></td>
        <td><center>3 -> 2</center></td>
        <td><center>0</center></td>
      </tr>
</table><br>

J'ai donc passé la notation allant de 1 à 5 en entier car elle était en objet

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C3314"> 3.3.1.4 Les doublons</a>

Seule la dataframe "df_review_br" est concernée :

<table>
      <tr>
        <th><center>Nom</center></th>
        <th><center>Nombre de lignes</center></th>
        <th><center>Commentaire</center></th>
      </tr>
      <tr>
        <td><center>df_review_br</center></td>
        <td><center>1 613 801  -> 1 611 202 (-2599 lignes ; -0.2%)</center></td>
        <td><center>Tous types de business</center></td>
      </tr>
</table><br>

Les répétitions des classes objets détectées dans les autres dataframes ne sembles pas incohérentes

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C3315">3.3.1.5 Outliers<a>   

Pas de souci apparement sur les types objets sur les 2 dataframes

##### <a href="Renard_Anthony_2_notebook_022023.ipynb#C33151">3.3.1.5.1 df_review_br<a>   

La colonne "stars" contient des valeurs entre 1 et 2 ; c'est ce que nous avions prévu :

<center><img alt="outliers" src="images/outliers/stars.png"/></center>

##### <a href="Renard_Anthony_2_notebook_022023.ipynb#C33122">3.3.1.5.2 df_final_api<a>  

La colonne "rating" contient des valeurs entre 1 et 2 ; c'est ce que nous avions prévu :

<center><img alt="outliers" src="images/outliers/rating.png"/></center>

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C3316"> 3.3.1.6 Rassemblement des dataframes</a>

Avant rassemblement des dataframes, nous avons le statut suivant : 
    
<table>
      <tr>
        <th><center>N°</center></th>
        <th><center>Nom</center></th>
        <th><center>Nombre de lignes</center></th>
        <th><center>Nombre de colonnes</center></th>
        <th><center>Nombre de restaurants</center></th>
      </tr>
      <tr>
        <td><center>1</center></td>
        <td><center>df_business</center></td>
        <td><center>52 286</center></td>
        <td><center>3</center></td>
        <td><center>52 286</center></td>
      </tr>
       <tr>
        <td><center>2</center></td>
        <td><center>df_review_br</center></td>
        <td><center>1 613 801</center></td>
        <td><center>3</center></td>
        <td><center>A filtrer</center></td>
      </tr>
       <tr>
        <td><center>3</center></td>
        <td><center>df_final_API</center></td>
        <td><center>37</center></td>
        <td><center>3</center></td>
        <td><center>33</center></td>
      </tr>
</table><br>

Je vais rassembler "df_business" (qui ne contient que des restaurants) et "df_business_br" qui contient tous les business ayant des notes entre 1 et 2 afin d'avoir "df_final_dataset" ne contenant que des restaurants avec des avis et des notes entre 1 et 2. Nous avons donc maintenant cette nouvelle dataframe (après avoir enlevé les doublons) :

<table>
      <tr>
        <th><center>N°</center></th>
        <th><center>Nom</center></th>
        <th><center>Nombre de lignes</center></th>
        <th><center>Nombre de colonnes</center></th>
        <th><center>Nombre de restaurants</center></th>
      </tr>
      <tr>
        <td><center>1</center></td>
        <td><center>df_final_dataset</center></td>
        <td><center>970 045</center></td>
        <td><center>3</center></td>
        <td><center>33 885</center></td>
      </tr>
</table><br>

Ajoutons maintenant les de "df_final_api". Nous obtenons donc :

<table>
      <tr>
        <th><center>N°</center></th>
        <th><center>Nom</center></th>
        <th><center>Nombre de lignes</center></th>
        <th><center>Nombre de colonnes</center></th>
        <th><center>Nombre de restaurants</center></th>
      </tr>
      <tr>
        <td><center>1</center></td>
        <td><center>df_final</center></td>
        <td><center>970 082</center></td>
        <td><center>3</center></td>
        <td><center>33 916</center></td>
      </tr>
</table><br>

#### <a href="Renard_Anthony_2_notebook_exploration_012023.ipynb#C3317">3.3.1.7 Bilan du nettoyage<a> 

Après toutes les opérations de nettoyage vues ci-dessus, voici la configuration du dataframe finale : <br> 


<table>
      <tr>
        <th><center>N°</center></th>
        <th><center>Nom</center></th>
        <th><center>Nombre de lignes</center></th>
        <th><center>Nombre de colonnes</center></th>
        <th><center>Type int64</center></th>
        <th><center>Type objet</center></th>
        <th><center>Nombre de restaurants</center></th>
      </tr>
      <tr>
        <td><center>1</center></td>
        <td><center>df_final</center></td>
        <td><center>970 082</center></td>
        <td><center>3</center></td>
        <td><center>1</center></td>
        <td><center>2</center></td>
        <td><center>33916</center></td>
      </tr>
</table><br>


Cette dataframe contient 970 082 avis et notes (1/5 et 2/5) de 33916 restaurants

<center><img alt="reataurant" src="images/restaurant.png"/></center>

Remarque : au vu du faible impact de la considération des données API sur la dataframe finale (faible quantité) et afin de garder une stabilité dans les données pour nos analyses (le résultat de l'API peut changer tous les jours), nous allons travailler avec df_final_dataset.

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C332"> 3.3.2 Images<a>

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C3321"> 3.3.2.1 Sélection des lignes et des colonnes d'un point de vue "métier"<a>

La problématique étant la suivante : "Labelliser automatiquement les photos postées", on va sélectionner les colonnes pertinentes répondants à notre problématique. On va donc ne garder que les colonnes "photo_id" et "label".  Voici le statut de df_images après ce filtrage :

<table>
      <tr>
        <th><center>N°</center></th>
        <th><center>Nom</center></th>
        <th><center>Nombre de lignes</center></th>
        <th><center>Nombre de colonnes</center></th>
      </tr>
      <tr>
        <td><center>1</center></td>
        <td><center>images</center></td>
        <td><center>200 100</center></td>
        <td><center>2</center></td>
      </tr>
</table><br>

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C3322"> 3.3.2.2 Les manquants<a>

Pas de manquants sur df_images.

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C3323"> 3.3.2.3 Les types<a>

Les types sont bien affectés sur df_images (que des objets).

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C3324"> 3.3.2.4 Les doublons</a>

Si l'on prend les 2 colonnes de "df_images", il n'y a pas de doublons. Cependant si l'on prend que "photo_id", on retrouve 2 "id" qui sont répétés. Ce qui veut dire que la même photo a 2 labels. On constate ici 2 fois la même photo. En vérifiant chaque photo, les 2 labels sont plutôt sur "drink" et "food" au lieu de "inside". 

<center><img alt="photo" src="images/photos/_CYoxbCIKuAwpq4crHCPWg.jpg"/></center>
<center><img alt="photo" src="images/photos/qtl5YDIc2q0yelW_e9FfqQ.jpg"/></center>

Je vais donc garder les 2 derniers éléments (drink et food)

<table>
      <tr>
        <th><center>Nom</center></th>
        <th><center>Nombre de lignes</center></th>
        <th><center>Commentaire</center></th>
      </tr>
      <tr>
        <td><center>df_images</center></td>
        <td><center>200100  -> 200098 (-2 lignes ; -0%)</center></td>
        <td><center>Doublons sur "photo_id" uniquement</center></td>
      </tr>
</table><br>

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C3325">3.3.2.5 Outliers<a>   

Pas de souci apparent sur les outliers "objets"

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C3326">3.3.2.6 Alignement de la liste d'images et le dataframe<a>   

Les "photos_id" de "df_images" sont toutes présentes de manière unique dans "list_images". On pourra donc exploiter toutes les images citées dans "df_images"

#### <a href="Renard_Anthony_2_notebook_exploration_012023.ipynb#C3327">3.3.2.7 Bilan du nettoyage<a> 

Après toutes les opérations de nettoyage vues ci-dessus, voici la configuration finale du dataframe : <br> 


<table>
      <tr>
        <th><center>N°</center></th>
        <th><center>Nom</center></th>
        <th><center>Nombre de lignes</center></th>
        <th><center>Nombre de colonnes</center></th>
        <th><center>Type objet</center></th>
      </tr>
      <tr>
        <td><center>1</center></td>
        <td><center>df_images</center></td>
        <td><center>200 098</center></td>
        <td><center>2</center></td>
        <td><center>2</center></td>
      </tr>
</table><br>

# <a href="Renard_Anthony_2_notebook_022023.ipynb#C4"> 4. Analyses <a>

## <a href="Renard_Anthony_2_notebook_022023.ipynb#C41"> 4.1 Texte <a>

Analyse sur "df_final_dataset" (sans les quelques lignes API pour garder une stabilité dans l'analyse); en effet à chaque requête via l'API, le nombre de commentaires et la notation peuvent évoluer.

### <a href="Renard_Anthony_2_notebook_exploration_022023.ipynb#C411"> 4.1.1 Analyses univariées <a>

#### <a href="Renard_Anthony_2_notebook_exploration_022023.ipynb#C4111"> 4.1.1.1 Notation <a>

<center><img alt="reataurant" src="images/analyse/notes_camenbert.png"/></center>

Il y a 58% de note à et 42% de notes à 2. Ce déséquilibre ne gêne pas puisque nous sommes dans un problème d'analyse textuelles des insatisfactions (avis)  et non de classification via ces notes. 

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C4112"> 4.1.1.2 Analyse de la longueur des avis <a>

Voici la répartition des longueurs des avis dans le corpus : 

<center><img alt="moustache" src="images/analyse/dispersion_length.png"/></center>
<center><img alt="moustache" src="images/analyse/moustache_lengh.png"/></center>

<table>
      <tr>
        <th><center>Nom</center></th>
        <th><center>Valeur</center></th>
      </tr>
      <tr>
        <td><center>Min</center></td>
        <td><center>1</center></td>
      </tr>
      <tr>
        <td><center>Max</center></td>
        <td><center>5000</center></td>
      </tr>
      <tr>
        <td><center>Moyenne</center></td>
        <td><center>656</center></td>
      </tr>
        <tr>
        <td><center>Médiane</center></td>
        <td><center>583</center></td>
      </tr>
        <td><center>Q1</center></td>
        <td><center>273</center></td>
       </tr>
        <tr>
        <td><center>Q2</center></td>
        <td><center>482</center></td>
       </tr>
        <tr>
        <td><center>Q3</center></td>
        <td><center>835</center></td>
       </tr>
</table>

La longueur des commentaires sont entre 1 et 5000. Cependant, 50% des commentaires ont entre 273 et 835 caractères.

Voici également le nombre d'avis par restaurant et du coup leur classement en terme de représentation dans le jeu de données :
<center><img alt="qte" src="images/analyse/avis_qte.png"/></center>

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C412"> 4.1.2 Analyses multivariées <a>

<center><img alt="corr" src="images/analyse/corr.png"/></center>
<center><img alt="pairplot" src="images/analyse/pairplot.png"/></center>
<center><img alt="pairgrid" src="images/analyse/pairgrid.png"/></center>

Pas de corrélation à constater entre le nombre de caractère et les notes 1 et 2 ; ceci n'a rien d'étonnant. 

Le restaurant le plus représenté dans le jeu de donnée est Mc-Donalds (un peu plus de 14000 avis -1.5% des avis)

## <a href="Renard_Anthony_2_notebook_022023.ipynb#C42"> 4.2 Images <a>

### <a href="Renard_Anthony_2_notebook_exploration_022023.ipynb#C421"> 4.2.1 Analyses univariées <a>

Voici la répartition de nos photos par label :

<center><img alt="label" src="images/analyse/labels_camenbert.png"/></center>

La répartition de nos labels ne sont pas équilibrés :<br>
* Food : 108 152 photos (54%)
* Inside : 56 029 photos (28%)
* Outside : 18 569 photos (9%)
* Drink : 15 670 photos (8%)
* Menu : 1678 photos (1%)

Total : 200 098 photos

Pour ce projet, nous prendrons un échantillon de 200 images par catégories. Nous aurons donc une répartition équilibrée pour le traitement (voir chapitre 6)

### <a href="Renard_Anthony_2_notebook_exploration_022023.ipynb#C422"> 4.2.2 Analyses multivariées <a>

Voici quelques photos pour chaque catégorie :

Food<br>
<img alt="food" src="images/analyse/food.png"/><br>
Inside<br>
<img alt="inside" src="images/analyse/inside.png"/><br>
Outside<br>
<img alt="outside" src="images/analyse/outside.png"/><br>
Drink<br>
<img alt="drink" src="images/analyse/drink.png"/><br>
Menu<br>
<img alt="menu" src="images/analyse/menu.png"/>

Pour un échantillon, le label semble être aligné à l'image indiqué. J'ai effectué cette vérification pour beaucoup d'images prises aléatoirement.

# <a href="Renard_Anthony_2_notebook_022023.ipynb#C5"> 5. Traitement des évaluations textuelles <a>

## <a href="Renard_Anthony_2_notebook_022023.ipynb#C51"> 5.1 Préparation avant traitement <a>

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C511"> 5.1.1 Définition des stopwords <a>

On va ici créer la liste de mots trop génériques pour être informatif (ils sont très utilisés dans le langage). Dans notre cas on utilisera les stopwords anglais selon NLTK.

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C512"> 5.1.2 Définition des mots anglais<a>

On va ici définir une variable contenant les mots anglais celon NLTK pour pouvoir conserver que les mots anglais lors de notre traitement 1.

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C513"> 5.1.3 Sélection d'un échantillon d'avis et création de raw corpus<a>

Après avoir récupéré l'ensemble des documents, j'en ai pris un échantillon (3000 documents) et j'ai créé le raw-corpus (corpus avant traitement). Cette manoeuvre va permettre de gagner un temps précieux dans le processing des traitements. Nous avons donc 2252 restaurants et 1 983 409 mots dans le raw-corpus. 

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C514"> 5.1.4 Création de la liste des adjectifs et des adverbes<a>

Pour le traitement LDA et NMF, je souhaite constater la différence de traitement dans le cas où je garde ou non les adjectifs et adverbes. Je crée donc ces 2 listes à l'image des 3000 documents sélectionnés.

## <a href="Renard_Anthony_2_notebook_022023.ipynb#C52"> 5.2 Traitement 1 : step by step <a>

J'ai traité ce raw-corpus de la manière suivantes :<br>
1. Passage en minuscule
2. Tokenisation
3. Suppression des stopswords
4. Suppression des mots rares
5. Suppression des mots ayant 3 lettres au plus
6. Conservation des mots 100% alphabétiques
7. Stemming (racinisation) 
8. Conservation des mots anglais

J'ai donc rédigé ces fonctions une par une et établi un pipeline qui appelle toutes ces fonctions en même temps et qui indique les étapes textuellement au fur et à mesure des traitements effectués  : 
<center><img alt="traitement" src="images/text/t1/traitement.png"/></center>
A la fin des traitements, le "raw-corpus" (1 983 409 mots) devient "corpus" (4196 mots) et je peux afficher le WordCloud suivant : 
<center><img alt="wc1" src="images/text/t1/wordcloud1.png"/></center>
Certains mots prennent plus de place que les autres et n'apportent pas forcément une indication claire sur les sujets d'insatisfaction. Par conséquent j'ai filtré sur le seuil d'apparition pour faire apparaitre ces sujets :
<center><img alt="wc2" src="images/text/t1/wordcloud2.png"/></center>

Voici la fréquence d'apparition des mots pour chaque WordCloud:<br>
Wordcloud1:<br>
<center><img alt="app" src="images/text/t1/pourcentage_mots.png"/></center>
"Food" apparait 2632 fois.<br>
Wordcloud2:<br>
<center><img alt="app" src="images/text/t1/pourcentage_mots_seuil.png"/></center>

Conclusion : on repère quelques sujets d’insatisfactions intéressants : 
Horribl « plate »,
Wrong,
Bland,
Differ,
Later.


## <a href="Renard_Anthony_2_notebook_022023.ipynb#C53"> 5.3 Traitement 2 : Latent Dirichlet Allocation (LDA) <a>

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C531"> 5.3.1 Traitement des documents "bruts" <a>

<center><img alt="lda1" src="images/text/t2/lda1.png"/></center>

Voici la fréquence d'apparition des mots dans chaque topic : 
<center><img alt="lda1" src="images/text/t2/hist_lda1.png"/></center>

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C532"> 5.3.2 Traitement des documents "bruts" avec suppression des adverbes et adjectifs <a>

<center><img alt="lda2" src="images/text/t2/lda2.png"/></center>

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C533"> 5.3.3 Traitement des documents traités (corpus) <a>

<center><img alt="lda3" src="images/text/t2/lda3.png"/></center>

Voici un traitement du corpus avec lda.model et une visualisation pyLDAvis :

<center><img alt="lda3" src="images/text/t2/lda3_avis.png"/></center>

On observe une différence entre le traitement du corpus par la fonction LDADERICHELET (utilisé pour les analyse LDA précédentes) et lda.model ci-dessus. 

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C534"> 5.3.4 Conclusion LDA <a>

Le traitement des documents "bruts" montre un résultat intéressant :
<center><img alt="lda1" src="images/text/t2/lda1_analyse.png"/></center>

## <a href="Renard_Anthony_2_notebook_022023.ipynb#C54"> 5.4 Traitement 3 : Negative Matrix Factorisation (NMF) <a>

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C541"> 5.4.1 Traitement des documents "bruts" <a>

<center><img alt="nmf1" src="images/text/t3/nmf1.png"/></center>

Voici la fréquence d'apparition des mots dans chaque topic : 
<center><img alt="nmf1" src="images/text/t3/hist_nmf1.png"/></center>

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C542"> 5.4.2 Traitement des documents "bruts" et suppression des adverbes et adjectifs<a>

<center><img alt="nmf2" src="images/text/t3/nmf2.png"/></center>

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C543"> 5.4.3 Traitement des documents traités (corpus)<a>

<center><img alt="nmf3" src="images/text/t3/nmf3.png"/></center>

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C543"> 5.4.4 Conclusion NMF<a>

Le traitement des documents "bruts" montre un résultat intéressant :

<center><img alt="nmf1" src="images/text/t3/nmf1_analyse.png"/></center>

## <a href="Renard_Anthony_2_notebook_022023.ipynb#C55"> 5.5 Conclusion <a>

Plusieurs tests (dont 3 LDA et 3 NMF) ont été effectués pour trouver le plus d’insatisfactions possible (et donc le meilleur résultat).  3 insatisfactions principales ressortent le plus lors du traitement des documents « brutes »  : le temps d'attente, la mauvaise cuisine, la mauvaise communication et comportement. 

# <a href="Renard_Anthony_2_notebook_022023.ipynb#C6"> 6. Traitement des images <a>

## <a href="Renard_Anthony_2_notebook_022023.ipynb#C61"> 6.1 Sélection d'un échantillon <a>

On a à notre disposition 200 098 photos. Pour mes traitements, je vais sélectionner 200 photos par catégorie (x5) soit 1000 photos : <br>
<center><img alt="ech" src="images/photos/ech.png"/></center>

## <a href="Renard_Anthony_2_notebook_022023.ipynb#C62"> 6.2 Scale Invariant Feature Transform (SIFT) <a>

Voici le plan d'actions pour l'utilisation de SIFT dans le cadre de la labélisation automatique des images :<br>
<center><img alt="plan" src="images/photos/SIFT/plan.png"/></center><br>

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C621"> 6.2.1 Quelques testes de transformations d'images <a>

Afin de savoir quels traitements je vais effectuer sur les images avant de les faire traiter par SIFT, je vais faire quelques transformations d'images pour constater l'impact sur elles d'un point de vu visuel. Je ferai ensuite le choix de garder ou non certains traitements.<br>
Dans un premier temps je vais choisir une photo que j'ai vais transformer en noir et blanc. Pour chacun d'elle, je vais égaliser l'histogramme. Voici les résultats : <br>
Image couleur :<br>
<center><img alt="ech" src="images/photos/transformation/col_eg.png"/></center><br>
Image noir et blanc :<br>
<center><img alt="ech" src="images/photos/transformation/nb_eg.png"/></center><br>

Pour chacun des résultats, on voit que les contrastes sont plus prononcés et donc les éléments de la photo sont plus mis en valeur. On remarque aussi avec les histogrammes cumulés normalisés, une pente linéaire un fois l'égalisation effectuée.

Sur l'image noir et blanc, j'ai ensuite ajouté un bruit Gaussien et un filtrage bruit par la moyenne 

<center><img alt="ech" src="images/photos/transformation/bruit.png"/></center><br>

On remarque que le lissage élimine le bruit mais apporte un flou sur l'image. 

Dans l'analyse SIFT suivante, je vais transformer mes images en noir et blanc égaliser l'histogramme pour mettre en valeur les contraste. J'ai aussi en parallèle ajouter l'élimination du bruit par lissage médian avant puis après l'égalisation mais cela a dégradé mes résultats finaux. 

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C622"> 6.2.2 Détermination et affichage des descripteurs d'une image <a>

Les descripteurs d'une image sont les points remarquables d'une image. Ils vont nous permettre de nous donner une indication sur son contenu. Pour l'image suivantes les descripteurs sont déterminés par de petits cercles : <br>
<center><img alt="plan" src="images/photos/SIFT/sift_descripteurs.png"/></center><br>

L'image contient 1106 descripteurs. Chaque descripteur est un vecteur de longueur 128.

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C623"> 6.2.3 Traitement des images <a>

#### <a href="Renard_Anthony_2_notebook_0220231.ipynb#C6231"> 6.2.3.1 Création des descripteurs de chaque image <a>

Pour notre échantillon de 1000 images j’ai 1 386 563 descripteurs <br>
(761 286 si application d’un filtre médian avant égalisation histogramme)<br>
(733 078 si application d’un filtre médian après égalisation histogramme)

#### <a href="Renard_Anthony_2_notebook_0220231.ipynb#C6232"> 6.2.3.2 Création des clusters de descripteurs <a>

2 choix s'offrent à nous. Soit je prends un nombre maximum de cluster soit la racine carrée du nombre de descripteur (SQRT( 1 386 563) = 1178 clusters); soit je prends le nombre minium qui est = nombre de classe attendu final x 10 = 50. Afin de ne pas perdre d'informations, je vais prendre le maximum de clusters (1178)* et je ferai ensuite une réduction de dimensions. J'utiliserai la fonction MiniBatchKmeans pour gagner du temps sur le temps de traitement.<br>
*(873 avec filtrage médian avant égalisation histogramme) <br>
*(856 avec filtrage médian après égalisation histogramme) 

#### <a href="Renard_Anthony_2_notebook_0220231.ipynb#C6233"> 6.2.3.3 Création des histogrammes/features des images <a>

Pour chaque image, on va prédire les numéros de cluster de chaque descripteur puis l'on va créer l'histogramme = comptage pour chaque numéro de cluster du nombre de descripteurs de l'image.<br>
Features d'une image = Histogramme d'une image = Comptage pour une image du nombre de descripteurs par cluster.<br>

#### <a href="Renard_Anthony_2_notebook_0220231.ipynb#C6234"> 6.2.3.4 Réduction de dimension <a>

Nous avons 1178 clusters pour 1000 images : dim = 1000 x 1178. On va donc d'abord faire une PCA en conservant une variance de 99%. On va ainsi réduire notre dimension en privilégiant la conservation de la structure globale du jeu de donnée et ainsi permettre un meilleur temps de traitement du T-SNE. Après PCA la dimension est dim = 1000 x 698. <br>
Après PCA, nous faisons une T-SNE qui permet de réduire encore plus notre jeu de donnée mais cette fois ci en mettant en valeur l'aspect local du jeu de donné. Après T-SNE, la dimension est dim= 1000 x 2.<br>
Remarque :<br>
-La réduction PCA permet de créer des features décorrélées entre elles, et de diminuer leur dimension, tout en gardant un niveau de variance expliquée élevé (99%)<br>
-L'impact est une meilleure séparation des données via le T-SNE et une réduction du temps de traitement du T-SNE

####  6.2.3.5 Résultats

<center><img alt="visu" src="images/photos/SIFT/analyse_visuelle.png"/></center><br>

Après réduction de dimension, on remarque que les catégories initialement fournies sont mélangées même si l'on observe une tendance sur les groupements. En utilisant Kmeans (à droite), mon classement est beaucoup plus segmenté. Cependant, le ARI score entre ces 2 classements est faible (15%). 

<center><img alt="visu" src="images/photos/SIFT/sift_analyse_classe.png"/></center><br>
<center><img alt="visu" src="images/photos/SIFT/class_report.png"/></center><br>

La matrice de confusion ci-dessus nous montre la corrélation entre les classes et les clusters. On observe que "outside" et "Menu" sont les correspondances les plus proches. Au globale on reste sur des scores assez faibles.

####  6.2.3.6 Conclusion

On voit ici que séparer les classes de façon non supervisée avec SIFT n'est si efficace pour notre cas. On a une piste intéressante mais on atteint certaines limites pour notre cas. On va donc tester une autre méthode : le CNN.

## <a href="Renard_Anthony_2_notebook_022023.ipynb#C63"> 6.3 Convolutional neural network (CNN) <a>

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C631"> 6.3.1  VGG16 : exploration <a>

Voici la structure du VGG16 : 

<center><img alt="cnn" src="images/photos/CNN/vgg16.png"/></center><br>

Dans Keras Application, VGG16 a été entraîné sur plus d’un million d’images provenant de la base de données ImageNet<br>
Chaque couche a un rôle précis :
- Convolution : son but est de repérer la présence d'un ensemble de features dans les images reçues en entrée
- Correction Relu : La couche de correction ReLU remplace donc toutes les valeurs négatives reçues en entrées par des zéros. Elle joue le rôle de fonction d'activation.
- L'opération de pooling consiste à réduire la taille des images, tout en préservant leurs caractéristiques importantes.
- Fully-connected permet de classifier l'image en entrée du réseau : elle renvoie un vecteur de taille N, où N  est le nombre de classes dans notre problème de classification d'image. Pour calculer les probabilités, la couche fully-connected multiplie donc chaque élément en entrée par un poids, fait la somme, puis applique une fonction d'activation (logistique si N=2, softmax si N>2 ) 



Ce CNN VGG16 est donc adapté à une classification en 1000 classes. Il va aider à répondre à la problématique (labélisation automatique des images) mais pas directement à la labélisation de nos images dans les 5 catégories que l'on désire : "inside", "outside", "food", "drink" et "menu". On fera donc du "transfert learning" pour l'adapter à notre besoin. Voici ci-dessous quelques prédictions faites par VGG16 :

<center><img alt="cnn" src="images/photos/CNN/conf1/image1.png"/></center><br>

Prédictions : 
- Oreille --> 21,8%
- Mais --> 21,7%
- Couperet --> 20%


<center><img alt="cnn" src="images/photos/CNN/conf1/image2.png"/></center><br>

Prédictions : 
- Restaurant --> 29%
- Planétarium --> 15%
- Train --> 12%

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C632"> 6.3.2 VGG16 : suppression de la dernière couche - apprentissage non-supervisé <a>

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C6321"> 6.3.2.1 Préparation du CNN et lecture des images <a>

On va ici faire du "transfert learning" en supprimant les dernières couches du modèle VGG16.<br>
<center><img alt="cnn" src="images/photos/CNN/conf2/vgg16_mod.png"/></center><br>

Cette opération s'effectue en mettant le paramètre "include_top" à FALSE. On a donc chargé VGG-16 pré-entraîné sur ImageNet et sans les couches fully-connected. On va donc récupérer la sortie de la couche "max_pooling". Cette sortie nous donnera directement les features (équivalent à la création d'histogramme pour SIFT) après lecture de toutes nos images. 

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C6322"> 6.3.2.2 Réduction de dimensions <a>

A la sortie de la couche "max pooling" nous avons un array de dimension : dim = 1000 x 25088. On va donc d'abord faire une PCA en conservant une variance de 99%. On va ainsi réduire notre dimension en privilégiant la conservation de la structure globale du jeu de donnée et ainsi permettre un meilleur temps de traitement du T-SNE. Après PCA la dimension est dim = 1000 x 917. <br>
Après PCA, nous faisons une T-SNE qui permet de réduire encore plus notre jeu de donnée mais cette fois ci en mettant en valeur l'aspect local du jeu de donné. Après T-SNE, la dimension est dim= 1000 x 2.<br>
Remarque :<br>
-La réduction PCA permet de créer des features décorrélées entre elles, et de diminuer leur dimension, tout en gardant un niveau de variance expliquée élevé (99%)<br>
-L'impact est une meilleure séparation des données via le T-SNE et une réduction du temps de traitement du T-SNE

#### 6.3.2.3 résultats 

<center><img alt="cnn" src="images/photos/CNN/conf2/resultat_cnn.png"/></center><br>

Après réduction de dimension, on remarque que les catégories initialement fournies apparaissent principalement segmentées. En utilisant Kmeans (à droite), mon classement est beaucoup plus segmenté. Cependant, le ARI score entre ces 2 classements est faible (36%) mais mieux qu'avec SIFT (15%)

<center><img alt="cnn" src="images/photos/CNN/conf2/heatmap.png"/></center><br>
<center><img alt="cnn" src="images/photos/CNN/conf2/class_report_cnn.png"/></center><br>

La matrice de confusion ci-dessus nous montre la corrélation entre les classes et les clusters. On observe que "drink" et "Menu" sont les correspondance les plus proches. Au globale on reste sur des scores assez faibles malgré une net amélioration vis à vis du classement fait par SIFT. 

#### 6.3.2.4 Conclusion

On voit ici que séparer les classes de façon non supervisée avec CNN VGG16 est mieux qu'avec SIFT mais l'efficacité reste critiquable. En effet, le réseau n'a pas été entrainé sur notre jeu d'images. On a donc une piste interessante mais on atteint certaines limites pour notre cas. Une solution serait donc d'entrainer notre réseau et lui demander seulement 5 classes. 

### <a href="Renard_Anthony_2_notebook_022023.ipynb#C633"> 6.3.3  VGG16 : apprentissage supervisé - transfert learning <a>

On va ici enlever les dernières couches du VGG16 comme dans la précédente étape puis je vais intégrer quelques couches qui vont me permettre d'avoir 5 catégories en sortie du réseau de neurones. On va donc garder les premières couches du VGG16 pré-entrainées et l'on va entrainer les dernières couches avec une parties de nos images.

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C6331"> 6.3.3.1  Configuration des paramètres principaux du modèle <a>

Cette partie permet de donner les chemins d'accès vers nos images mais aussi de définir quelques paramètres (ex : taille des images, activation de l'entrainement, l'EPOCH...)

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C6332"> 6.3.3.2   Création des jeux de test, entrainement et validation <a>

Nous avons un peu moins de 200 000 photos disponibles dans notre jeu de données. On pourrait utiliser ces 200 000 photos pour réaliser nos 3 sections de données mais le temps de traitement sera extrêmement long. Par conséquent je vais prendre qu'un échantillon de ces photos (sans prendre en compte les 1000 photos choisis dans les études précédentes pour le jeu d'entrainement). Pour le jeu de test, je vais prendre les 1000 photos de notre échantillon utilisé précédemment pour pouvoir comparer les performances entre elles sur le même jeu de données.<br>
Prenons un échantillon de 5000 photos pour le jeu d'entrainement et validation et 1000 pour le jeu de test et regardons la performance de notre CNN.

<center><img alt="cnn" src="images/photos/CNN/conf3/split.png"/></center><br>

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C6333"> 6.3.3.3 Préparation du CNN et entrainement <a>

<center><img alt="cnn" src="images/photos/CNN/conf3/vgg16_mod.png"/></center><br>

Après avoir supprimé les dernières couches du VGG16 d'origine, j'ai ajouté les couches suivantes :<br>
-Flatten : aplatissement de l'image de sortie en un vecteur<br>
-Dropout : régularisation pour éviter l'overfitting<br>
-Dense = Fully-Connected + Softmax : ajour de la couche de sortie à 5 neurones pour la prédiction (probabilistique)


La configuration totale du VGG16 modifiée est la suivante :<br>
<center><img alt="cnn" src="images/photos/CNN/conf3/conf_vgg16_mod.png"/></center><br>

J'entraine maintenant le VGG16 modifié avec ses nouvelles couches. Voici le résultat : <br>
<center><img alt="cnn" src="images/photos/CNN/conf3/EPOCH_10_TRAIN_5000_TEST_1000_ECH/results.png"/></center><br>
<center><img alt="cnn" src="images/photos/CNN/conf3/EPOCH_10_TRAIN_5000_TEST_1000_ECH/results2.png"/></center><br>
<center><img alt="cnn" src="images/photos/CNN/conf3/EPOCH_10_TRAIN_5000_TEST_1000_ECH/curves.png"/></center><br>

Cette approche génère un résultat très intéressant au vu de la valeur de "accuracy" et "loss".<br>
Remarque : test = jeu de validation

#### <a href="Renard_Anthony_2_notebook_022023.ipynb#C6334"> 6.3.3.4 Prédictions <a>

Suite à la création du nouveau modèle, j'ai donc effectué la prédiction sur le jeu de test pour constater sa performance. Avant d'observer ces images, nous devons trouver la correspondance suivante :<br>
<center><img alt="cnn" src="images/photos/CNN/conf3/EPOCH_10_TRAIN_16000_TEST_4000/classes.png"/></center><br>

Voici maintenant quelques prédictions : 

<center><img alt="cnn" src="images/photos/CNN/conf3/EPOCH_10_TRAIN_5000_TEST_1000_ECH/predicts2.png"/></center><br>

Au global, on voit que le modèle a fait de bonnes prédictions malgré quelques erreurs de temps en temps dû au taux d'erreur mais aussi en pratique, il n'est pas simple de différencier un menu avec une photo de "food" dessus. Pour notre cas, le ARI SCORE atteint 45%.

# 7. Synthèse

Dans un premier temps j'ai chargé les données images et textuelles via les éléments disponibles (dataset + image(jpg) + API YELP).<br>
Ensuite j'ai exploré les données en analysant le contenu des données (fond et forme) et en les nettoyants (EDA : Exploratory Data Analysis).<br>

Concernant les données textuelles, j'ai utilisé 3 méthodes : <br><br>
-Méthode 1 : le step by step. Pour cette méthode j'ai effectué différents traitements : création du corpus, passage en minuscule, tokenisation, suppression des stopswords, suppression des mots rares, suppression des mots ayant 3 lettres au plus, conservation des mots 100% alphabétiques, stemming (racinisation) et conservation des mots anglais. Cette méthode est intéressante car elle a permis de relever des "mots" reflétant l'insatisfaction des clients.<br>
-Méthode 2 : Latent Dirichlet Allocation (LDA) : en prenant en entrée le corpus non traité, le corpus non traité - adverbes et adjectifs et enfin le corpus traité en méthode 1. <br>
-Méthode 3 : Non-negative matrix factorization (NMF) en prenant en entrée le corpus non traité, le corpus non traité - adverbes et adjectifs et enfin le corpus traité en méthode 1<br>
Les 2 dernières méthodes sont très intéressantes car elles regroupent les mots en clusters. En utilisant le corpus non traité, j'ai relevé 4 sujets d'insatisfactions : la mauvaise communication, le temps d'attente, le mauvais comportement du personnel et la mauvaise cuisine.<br>
LDA et NMF sont les 2 méthodes les plus performantes pour offrir une solution à notre problématique. 

Concernant les données visuelles, j'ai utilisé 4 méthodes : <br><br>
-Méthode 1 : Scale-invariant feature transform (SIFT) : méthode très intéressante qui a offert une performance ARI de 15%<br>
-Méthode 2 : Convolutional neural network(CNN) VGG16 : cette méthode est intéressante et va servir à nous aider à identifier les labels des photos. Cependant il faudra faire du "transfert learning" pour l'adapter à notre problématique.<br>
-Méthode 3 : VGG16 en apprentissage non supervisé + transfert learning : cette solution nous offre un ARI-SCORE de 35% soit +de 50% que le SIFT; cependant ce score reste faible.<br>
-Méthode 4 : VGG16 en apprentissage supervisé + transfert learning : ici nous allons modifier et entrainer CNN VGG16 (plusieurs configurations testées). Avec cette méthode et pour un jeu d'entrainement de 5000 images, nous avons un accuracy de 89% et un Loss de 0.43. Un test sur l'échantillon de 1000 photos utilisé précédemment nous donne un ARI score = 45%. C'est la méthode la plus performante des 4.<br> Remarque : un jeu d'entrainement de 1000 images donne un accuracy = 79%, loss = 2.34 et ARI SCORE = 0.29. On peut donc pressentir qu'une augmentation du jeu d'entrainement entraine une amélioration de la performance de notre modèle.<br><br>

Opportunités :
- Pour les données textuelles, on pourrait mettre en place un Recurrent Neural Network (RNN) et constater la performance
- Pour les données visuelles, on pourrait tenter d'améliorer la performance de la méthode 4 en augmentant encore le nombre d'échantillons sur le jeu d'entrainement et/ou en augmentant le nombre de couches dans le CNN. On pourrait aussi mettre en place un réseau de neurones personnalisé ou utiliser d’autres modèles existant (ex : VGG19)

# FIN